In [3]:
import pysam as ps
import numpy as np
import collections
import matplotlib.pyplot as plt
import pysam as ps
import numpy as np
from collections import defaultdict
from scipy.ndimage.measurements import label, standard_deviation
from scipy.stats.mstats_basic import kstest, normaltest
from sklearn.cluster import KMeans
import sys
from scipy.signal import savgol_filter
import math
from subprocess import call
import os.path
from utils import Gene, TSS, Point
from scipy import stats
from sklearn import svm
import sympy
import math
from math import e
import random
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Input,Conv2D,Activation,Dense,Lambda,Flatten,Embedding,PReLU,BatchNormalization,Bidirectional,LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold
from tensorflow.keras import backend as K
import copy
from sklearn.utils import shuffle
dic = {
    'gene':'./data/gene/GRCh37.gene.bed',
    'non_gene':'./data/gene/non_gene.bed',
    'non_gene_4':'./data/gene/non_gene_4.bed',
    'non_gene_2':'./data/gene/non_gene_2.bed',
    'fasta':'/home/jiay/Desktop/hg19/hg19.fa',
    'bam1':'./data/051.bam',
    'bam2':'./data/051_2.bam',
    'bam3':'./data/051_3.bam',
    'bam123':'./data/051_123.bam',
    'bam234':'./data/051_234.bam',
    'bam4':'./data/051_4.bam',
    'bam5':'./data/051_5.bam',
    'TSS_low':'./data/gene/low_expressed.bed',
    'TSS_HK':'./data/gene/HK.bed',
    'TSS_silent':'./data/gene/silent_gene_TSS.bed',
    'ATAC_hema_1':'./data/gene/ATAC_hema_1.bed',
    'ATAC_hema_2':'./data/gene/ATAC_hema_2.bed',
    'ATAC_Bcell':'./data/gene/ATAC_bcell.bed',
    'ATAC_Brain':'./data/gene/ATAC_brain.bed',
    'ATAC_liver':'./data/gene/ATAC_liver.bed',
    'ATAC_colon':'./data/gene/ATAC_colon.bed',
    'ATAC_heart':'./data/gene/ATAC_heart.bed',
    'ATAC_stomach':'./data/gene/ATAC_stomach.bed',
    'ATAC_lung':'./data/gene/ATAC_lung.bed',
    'ATAC_skin':'./data/gene/ATAC_skin.bed',
    'ATAC_leukocyte':'./data/gene/ATAC_leukocyte.bed',
    'ATAC_hema':'./data/gene/ATAC_hema.bed',
    }

ImportError: cannot import name 'get_config' from 'tensorflow.python.eager.context' (/share/home/jiayren/.conda/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/eager/context.py)

In [33]:
TSS_test = []
# with open(dic['ATAC_liver'],'r') as f:
#     for line in f:
#         ll = line.strip().split('\t')
#         if ll[-1] in ['6'] and ll[0] == '1':
#             TSS_test.append(TSS(ll[0], int(ll[1])))
with open(dic['TSS_HK'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] in ['1']:
            TSS_test.append(TSS(ll[0], int(int(ll[1])+1000)))
# with open(dic['ATAC_hema'],'r') as f:
#     for line in f:
#         ll = line.strip().split('\t')
#         if ll[0] in ['1']:
#             TSS_test.append(TSS(ll[0], int((int(ll[1])+int(ll[2]))/2)))
# with open(dic['non_gene'],'r') as f:
#     for line in f:
#         ll = line.strip().split('\t')
#         TSS_test.append(TSS(ll[0], int(1000+int(ll[1]))))
# TSS_test = TSS_test[:300]
TSS_test = shuffle(TSS_test)
TSS_test = TSS_test[:300]


In [34]:
test_data = []
testbam = ps.AlignmentFile(dic['bam123'])#lstm
test_matrix = []
up = 1000
down = 1000
# chrom = '1'
for j, tss in enumerate(TSS_test):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    up_end = np.zeros(up+down, dtype= int)
    down_end = np.zeros(up+down, dtype= int)
    long = np.zeros(up+down, dtype= int)
    short = np.zeros(up+down, dtype= int)
    cov = np.zeros(up+down, dtype= int)
    wps = np.zeros(up+down, dtype=float)
    win = 120
    for r in testbam.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            ss = r.reference_start - start
            ee = r.reference_start - start + abs(r.isize)
            if ss >= 0:
                up_end[ss] += 1
            else:
                ss = 0
            if ee < end - start:
                down_end[ee] += 1
            else:
                ee = end - start
            for i in range(ss, ee):
                cov[i] += 1
            if 200 >= abs(r.isize) > 130:
                for i in range(ss, ee):
                    long[i] += 1
            if abs(r.isize) <= 130:
                for i in range(ss, ee):
                    short[i] += 1
            # wps_total
            region1 = int(max(0, ss + win/2))
            region2 = int(min(ee - win/2, end-start))
            i = region1
            while i < region2:
                wps[i] += 1
                i = i+1
            # wps_part
            region1 = int(max(0, ss - win/2))
            region2 = int(min(end-start, ss + win/2))
            i = region1
            while i < region2:
                wps[i] -= 1
                i = i + 1
            # wps_part
            region1 = int(max(ee - win/2, 0))
            region2 = int(min(ee + win/2, end-start))
            i = region1
            while i < region2:
                wps[i] -= 1
                i = i+1
    k = 0
    win = 40
    feature_win = np.zeros((int((up+down)/win), 4), dtype= int)
    while k < (up+down)/win:
        ss = k * win
        ee = k * win + win
        ff = []
        ff.append(int(round(np.mean(cov[ss:ee]))))
        ff.append(int(round(np.mean(long[ss:ee]-short[ss:ee]))))
        ff.append(int(round(np.sum(abs(up_end[ss:ee]-down_end[ss:ee])))))
        ff.append(int(round(np.mean(wps[ss:ee]))))
        feature_win[k] = np.array(ff)
        k = k + 1
    test_matrix.append(feature_win)
test_matrix = np.array(test_matrix)

lstm_test = []
for mat in test_matrix:
    lstm_test.append(mat)
lstm_test = np.array(lstm_test)
'''cnn_input'''
test_data = []
for j, tss in enumerate(TSS_test):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
#    distribution_matrix = np.zeros((int(up+down), 200), dtype=int)
    distribution_matrix = np.zeros((200,200),dtype=int)
    for r in testbam.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse and 50 < abs(r.isize) < 250:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            if r.reference_start < start:
                continue
            if r.reference_start + abs(r.isize) > end:
                continue
            ss = max(0, r.reference_start - start)
            relative_isize = abs(r.isize)-50
            distribution_matrix[ss//10,relative_isize] += 1
    test_data.append(distribution_matrix)
test_data = np.array(test_data)

cnn_test = []
for mat in test_data:
    cnn_test.append(mat)
cnn_test = np.array(cnn_test)


In [6]:
keras.backend.clear_session()

In [37]:
model_a = load_model('./model/model_a_hema.h5', compile=False)
model_b = load_model('./model/model_b_hema.h5', compile=False)
re_a = model_a([cnn_test,lstm_test]).numpy()
re_b = model_b([cnn_test,lstm_test]).numpy()
# print(re_a)
re = (re_a + re_b) / 2
# re.sort(axis=0)
# plt.plot(np.arange(0,len(re),1),re, label='OCRfinder')
# plt.show()
count = 0
for rr in re:
    if rr >= 0.5:
        count += 1
print(round(count/len(re_a)*100,2))

95.33
